<a id="functions"></a>
# WebScraping

El web scraping (“raspado” de páginas web) consiste en la extracción de los datos significativos de una o varias páginas web determinadas, o de todas las páginas web que estén relacionadas mediante enlaces en un sitio web, para una manipulación o análisis posterior .

Esta extracción se realiza obteniendo la información a través de "eliminar" la información que no nos interesa, los metadatos y quedarnos solo con los datos que nos interesan, los datos puros.

Los datos están inmersos dentro de la maraña de información que es el HTML y debemos navegar e inspeccionar estos datos y encontrar patrones de funcionamiento. Estos patrones se repiten de forma estructurada, y nosostros debemos aplicar esos patrones para separar el "grano" (datos) de la "paja"

Para hacer estos ejercicios es necesario que repaseis los recursos disponibles en el tema sobre WebScraping, y las librerias que vamos a usar.
En el repositorio público del módulo tenéis disponibles [estos notebooks](https://github.com/jssdocente/LMSGI-2122/tree/main/_transversal/python/notebooks/2.%20Webscraping) y unos [ejemplos prácticos](https://github.com/jssdocente/LMSGI-2122/blob/main/_transversal/python/notebooks/2.%20Webscraping/2.5%20Ejemplos%20pr%C3%A1cticos%20de%20web%20scraping.ipynb) muy interesantes y de gran valor de aprendizaje.

También teneís disponbile este [documento](https://docs.google.com/document/d/1CJ7MMTkvvtuAhw02YrhRaaSyO7sj-n03wOS9E2Fs2Ko/edit?usp=sharing) qué explica los conceptos más importantes.


## 1. Obtener los valores de la Bolsa de Madrid

Para este ejercicios inspeccionaremos la [web de la Bolsa de Madrid](http://www.bolsamadrid.es/esp/aspx/Indices/Resumen.aspx).
Este ejercicio lo vamos a dividir en varias partes, hasta conseguir el resultado final.

### 1.1. Obtener el nombre de los índices que componen la bolsa de Madrid.

El objetivo es obtener una lista, e imprimir esa lista con los nombres de los índices que la componen.<br>
Para obtenerla necesitamos acceder a esta página donde se encuentra un resumen de los índices, su nombre y una serie de datos adicionales.


In [1]:
# definir la función que devuelva los nombres de los índices de la Bolsa de Madrid

# importar librerias necesarias. En el resto de ejercicios, importar las necesarias, no siempre serán estas, podrán requerirse adicionales.
import requests
from bs4 import BeautifulSoup

def obtenerListaIndicesBolsaMadrid():
    url = "https://www.bolsamadrid.es/esp/aspx/Indices/Resumen.aspx"

    # completar el código necesario
    page = requests.get(url)
    soap = BeautifulSoup(page.content)

    table =soap.find('table',{'id': 'ctl00_Contenido_tblÍndices'})
    #print(table)

    listaIndices = []

    for fila in table.find_all('tr'):
        celdas = fila.find_all('td')
        if len(celdas)>0:
            listaIndices.append(celdas[0].string)

        
    return listaIndices

In [ ]:
# Probar la funcion
obtenerListaIndicesBolsaMadrid()

### 1.2. Obtener un diccionario con los nombres de los índices que componen la bolsa de Madrid.

El objetivo es obtener un diccionario, donde la clave sea el nombre del índice y su valor, el máximo historio que ha tenido<br>
Ese diccionario obtenido lo debemos ordenar de mayor a menor según su máximo valor.<br>
Por último imprimirlo.

In [3]:
# definir la función que devuelva los nombres de los índices de la Bolsa de Madrid
def obtenerDiccioinarioIndicesBolsaMadrid():
    url = "https://www.bolsamadrid.es/esp/aspx/Indices/Resumen.aspx"

    # completar el código necesario para obtener diccionario
    page = requests.get(url)
    soap = BeautifulSoup(page.content)

    table =soap.find('table',{'id': 'ctl00_Contenido_tblÍndices'})
    #print(table)

    numFila=0
    dicIndices = {}

    for fila in table.find_all('tr'):
        celdas = fila.find_all('td')
        if len(celdas)>0:
            dicIndices.update({celdas[0].string: float(celdas[4].string.replace('.','').replace(',','.') ) })

    # ordenar por máximo
    return sorted(dicIndices.items(), key=lambda x:x[1])



In [ ]:
# Probar la funcion
obtenerDiccioinarioIndicesBolsaMadrid()

### 1.3. Crear un dataframe (pandas) con ciertas columnas de los índices la bolsa de Madrid.

El objetivo es dar un paso más, y obtener un dataframe con las columnas (indice, anterior, ultimo, maximo, minimo). La columna clave será el nombre del índice<br>
Por último imprimirlo.
Para la realización de este ejercicio os podeís guiar por este [Extración del texto de un discurso](https://github.com/jssdocente/LMSGI-2122/blob/main/_transversal/python/notebooks/2.%20Webscraping/2.5%20Ejemplos%20pr%C3%A1cticos%20de%20web%20scraping.ipynb).

In [5]:
# definir la función que resuelva el ejercicio
import pandas as pd

def obtenerDataframeIndicesBolsaMadrid():
    url = "https://www.bolsamadrid.es/esp/aspx/Indices/Resumen.aspx"

    # completar el código necesario para obtener diccionario
    page = requests.get(url)
    soap = BeautifulSoup(page.content)

    table =soap.find('table',{'id': 'ctl00_Contenido_tblÍndices'})
    #print(table)

    numFila=0
    datos = {"indice": [], "anterior": [],"ultimo": [],"maximo": [], "minimo": [] }
    
    
    for fila in table.find_all('tr'):
        celdas = fila.find_all('td')
        if len(celdas)>0:
            indiceNombre = celdas[0].string
            antValor = convertValueToFloat(celdas[1].string)
            ultValor = convertValueToFloat(celdas[2].string)
            maxValor = convertValueToFloat(celdas[4].string)
            minValor = convertValueToFloat(celdas[5].string)
            datos["indice"].append(indiceNombre)
            datos["anterior"].append(antValor)
            datos["ultimo"].append(ultValor)
            datos["maximo"].append(maxValor)
            datos["minimo"].append(minValor)

            #datos.update({indiceNombre : [indiceNombre, antValor, ultValor, maxValor,minValor]})

    # Crear el dataframe de pandas
    dt = pd.DataFrame(datos)
    
    #dt.set_index('indice')

    return dt


def convertValueToFloat(strValue):
    return float(strValue.replace('.','').replace(',','.'))

In [ ]:
# Probar la funcion
obtenerDataframeIndicesBolsaMadrid()

### 1.4. Obtener los nombres de las Acciones del índice IBEX

En este ejercicio se requiere obtener una lista de las acciones que conforman el índice del IBEX-35.<br>
Al igual que el ejercicio 1.1, los nombres de devuelven en forma de lista

In [ ]:
# definir la función que devuelva los nombres de los índices de la Bolsa de Madrid
def obtenerListaAccionesIBEX():
    url = "https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000&punto=indice"

    # completar el código necesario

In [ ]:
# Probar la funcion
obtenerListaAccionesIBEX()

### 1.5. Obtener un diccionario con el nombre y el volumen de la sesión

El objetivo es obtener un diccionario, donde la clave sea el nombre de la acción y el volmen movido durante la sesión<br>
Ese diccionario obtenido lo debemos ordenar de mayor a menor según su volumen.<br>
Por último imprimirlo.

In [ ]:
# definir la función resuelve el ejercicio
def obtenerDiccionarioListaAccionesIBEX():
    url = "https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000&punto=indice"

    # completar el código necesario

In [ ]:
# Probar la funcion
obtenerDiccionarioListaAccionesIBEX()

### 1.6. Crear un dataframe (pandas) con ciertas columnas de las acciones.

El objetivo es dar un paso más, y obtener un dataframe con las columnas (fecha, nombre, precio, max, min, volumen). La columna clave será el nombre de la acción<br>
El orden de las columnas en el dataframe deben ser las siguientes: *fecha, nombre, precio, max, min, volumen*.<br>
Por último imprimirlo.

In [ ]:
# definir la función que resuelva el ejercicio
def obtenerDataframeAccionesIBEX():
    url = "https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000&punto=indice"

    # completar el código necesario para obtener diccionario




    # ordenar por máximo



In [ ]:
# Probar la funcion
obtenerDataframeAccionesIBEX()

### 1.7. Guardar el dataframe obtenido en un fichero CSV

El objetivo es dar un paso más, y guardar los valores obtenidos en el dataframe en un fichero CSV (separado por , o ; o ...)<br>
Este ejercicio es una continuación del ejercicio 1.6, con lo que se creará otra función para guardar el dataframe obtenido en un fichero CSV.
El orden de las columnas debe ser el siguiente: *fecha, nombre, precio, max, min, volumen*.<br>
 
Este fichero si no existe se crea, y si existe, se le anexarán las nuevas filas para los datos de la sesión para ese día.
Por último imprimirlo.

¿Ayuda? podeis seguir este [articulo](https://www.delftstack.com/es/howto/python-pandas/write-a-pandas-dataframe-to-csv/) donde explica cómo guardar un dataframe a un CSV.

In [ ]:
# definir la función que resuelva el ejercicio. 

#Se el pasan 2 argumentos, el dataframe(df) y el nombre del fichero CSV donde se va a guardar
csvfile = "datosAccionesIbex.csv"

def saveToCSV(df, csvFileName):
    
    # completar el código necesario para obtener diccionario




    # ordenar por máximo
    


In [ ]:
# Probar la funcion

obtenerDataframeAccionesIBEX()



saveToCSV()